In [51]:
pip install swifter

In [52]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from shapely.ops import nearest_points
from shapely.geometry import MultiPoint
from geopy.distance import distance
import swifter

# Load your coordinate points from CSV (replace with your actual CSV file path)
df = pd.read_csv('/content/10_box_benchmark.csv')
df

,Longitude,Latitude,EMAG2v3,cm_curie_point_depth,wgm2012_freeair_ponc,gl_elevation,rayleigh_group,sc_crust_den,interpolated_bouguer,igrf_dec,love_phase,gl_tot_sed_thick
0,24.048,-88.826444,-49.260544,55.956760,-45.722305,2762.8020,3.926312,2788.8086,-1.748929,-49.767223,4.750790,0.0000
1,24.144,-88.826444,-49.069367,55.693024,-45.591404,2763.8020,3.926312,2788.8083,-1.607633,-49.861222,4.750790,0.0000
2,24.240,-88.826444,-49.478714,55.410805,-45.459830,2763.8020,3.926312,2788.7847,-1.457480,-49.955225,4.750790,0.0000
3,24.336,-88.826444,-50.969139,54.957836,-45.264020,2764.8020,3.926312,2788.7244,-1.299575,-50.049233,4.750790,0.0000
4,24.432,-88.826444,-52.450505,54.471840,-45.068275,2765.8020,3.926312,2788.6482,-1.132913,-50.143247,4.750790,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
356906,81.552,88.826444,6.042610,12.265153,12.249834,-4299.0000,3.757340,2943.9272,424.319586,48.848612,4.634415,1779.5532
356907,81.648,88.826444,5.311905,12.314054,12.248323,-4298.8877,3.757340,2943.9272,424.438085,48.930633,4.634415,1779.3741
356908,81.744,88.826444,4.571694,12.386601,12.312545,-4298.6377,3.757340,2943.9270,424.587842,49.012643,4.634415,1779.0112
356909,81.840,88.826444,3.831456,12.458186,12.433552,-4298.3315,3.757340,2943.9270,424.775682,49.094643,4.634415,1778.6813


In [53]:
# Assuming your CSV has columns 'Latitude' and 'Longitude'
points = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
df['points'] = points

In [54]:
df['points']

0         POINT (24.048 -88.8264436099539)
1         POINT (24.144 -88.8264436099539)
2          POINT (24.24 -88.8264436099539)
3         POINT (24.336 -88.8264436099539)
4         POINT (24.432 -88.8264436099539)
                        ...               
356906     POINT (81.552 88.8264436099539)
356907     POINT (81.648 88.8264436099539)
356908     POINT (81.744 88.8264436099539)
356909      POINT (81.84 88.8264436099539)
356910     POINT (81.936 88.8264436099539)
Name: points, Length: 356911, dtype: object

In [55]:
# Define the boundary boxes for selection
boundary_boxes = [
    # USA: (latitude range: 24N to 49N, longitude range: 125W to 66W)
    (24, -125, 49, -66),    # Box 1: from (24N, 125W) to (49N, 66W)

    # Brazil: (latitude range: 5N to 34S, longitude range: 74W to 35W)
    (5, -74, -34, -35),     # Box 2: from (5N, 74W) to (34S, 35W)

    # Australia: (latitude range: 10S to 44S, longitude range: 113E to 154E)
    (-10, 113, -44, 154),   # Box 3: from (10S, 113E) to (44S, 154E)

    # Canada: (latitude range: 49N to 83N, longitude range: 141W to 52W)
    (49, -141, 83, -52),    # Box 4: from (49N, 141W) to (83N, 52W)

    # Argentina: (latitude range: 22S to 55S, longitude range: 73W to 53W)
    (-22, -73, -55, -53),   # Box 5: from (22S, 73W) to (55S, 53W)

    # China: (latitude range: 18N to 53N, longitude range: 74E to 135E)
    (18, 74, 53, 135),      # Box 6: from (18N, 74E) to (53N, 135E)

    # Russia: (latitude range: 41N to 82N, longitude range: 30E to 180E)
    (41, 30, 82, 180),      # Box 7: from (41N, 30E) to (82N, 180E)

    # South Africa: (latitude range: 22S to 35S, longitude range: 16E to 33E)
    (-22, 16, -35, 33),     # Box 8: from (22S, 16E) to (35S, 33E)

    # India: (latitude range: 8N to 37N, longitude range: 68E to 97E)
    (8, 68, 37, 97),        # Box 9: from (8N, 68E) to (37N, 97E)

    # Greenland: (latitude range: 60N to 84N, longitude range: 20W to 75W)
    (60, -75, 84, -20)      # Box 10: from (60N, 75W) to (84N, 20W)
]



# Function to check if a point is within any boundary box
def is_within_boundary_box(point, box):
    min_longitude, min_latitude, max_longitude, max_latitude = box
    if (min_longitude < point.x < max_longitude and min_latitude < point.y < max_latitude):
      return True

    return False


# Calculate distance to the closest coastline point for each coordinate
def label_by_zone(point, bounding_boxes):
  for i, box in enumerate(bounding_boxes):
    if is_within_boundary_box(point, box):
      return i


  return -1



# Create a new column 'Distance_to_Coast' in your DataFrame
df['zone_code'] = df['points'].swifter.apply(lambda row: label_by_zone(row, boundary_boxes))

# Optionally, you may want to save or further process the DataFrame with distances
df.to_csv('output_with_distances.csv', index=False)

Pandas Apply:   0%|          | 0/356911 [00:00<?, ?it/s]

In [56]:
df['zone_code'].value_counts()

zone_code
6    166071
9     89910
3     59995
0     17940
8     11711
5     11284
Name: count, dtype: int64

In [57]:
import xarray as xr
import numpy as np

processed_df= xr.DataArray( df['zone_code'].values , dims=['lat','lon'], coords= {'lat': df['Latitude'].values.flatten(), 'lon': df['Longitude'].values.flatten()})

processed_df['__xarray_dataarray_variable__'].plot()

ValueError: different number of dimensions on data and dims: 1 vs 2